# Formål

Script scraper diverse konsulenthuses sider for artikler og indlæg omkring kunstig intelligens (AI, artificial intelligence).

**Liste over konsulenthuse**
- (x) Ernst & Young (EY) (en del af BIG4 i accounting) https://www.ey.com/en_gl
- Deloitte. (en del af BIG4 i accounting) [https://www2.deloitte.com](https://www2.deloitte.com/)
- (x) KPMG. (en del af BIG4 i accounting) https://home.kpmg/xx/en/home.html
- PricewaterhouseCoopers (PwC) (en del af BIG4 i accounting) https://www.pwc.com/
- (x) McKinsey (Top Management consulting) https://www.mckinsey.com/
- (x) Boston consulting group (Top Management consulting) https://www.bcg.com/
- Bain & Company (Top Management consulting) https://www.bain.com/
- Accenture (IT consulting - verdens største konsulenthus) https://www.accenture.com/us-en
- Capgemini (IT consulting nr. 2 mener jeg) https://www.capgemini.com/
- Tata consulting (IT consulting - nr. 3 mener jeg målt på omsætning) https://www.tcs.com/
- IBM (Det er ikke sikkert, men de har vel Watson og måske interessant?) [https://www.ibm.com](https://www.ibm.com/)

# Bain URLs

Search for "artificial intelligence" filtered to include articles and brief, sorted by date: https://www.bain.com/search/?searchValue=artificial+intelligence&filters=%7Ctypes%28426%2C427%29&pageNumber=0&sortValue=date


### Libraries

Importing libraries...

In [1]:
import scrapy
import requests
from scrapy import Selector

import os
import sys
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains

import time

### URL list

List of URLs is extracted from search for "artificial intelligence" filtered to include articles and brief, sorted by date: https://www.bain.com/search/?searchValue=artificial+intelligence&filters=%7Ctypes%28426%2C427%29&pageNumber=0&sortValue=date

In [2]:
start_url = "https://www.bain.com/search/?searchValue=artificial+intelligence&filters=%7Ctypes%28426%2C427%29&pageNumber=0&sortValue=date"

In [11]:
## Scrape article urls (convert to code to use)

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.chrome.options import Options

chromeOptions = Options()
chromeOptions.add_argument("--kiosk")

driver = webdriver.Chrome(executable_path = 'C:\\bin\\chromedriver', options=chromeOptions)

driver.get(start_url)
time.sleep(2)
driver.find_element_by_css_selector('#cooky_policy > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > a:nth-child(1)').click()
time.sleep(2)

pageSel = Selector(text = driver.page_source)

urls = pageSel.css('div.card__content > a::attr(href)').extract()
titles = title_test = pageSel.css('div.card__content > a > h5::text').extract()

urldict = {}
urldict['urls'] = urls
urldict['titles'] = titles

coordinates = {'x': 0, 'y': 1600}
driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))

while True:
    try:
        wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//span[@class='icon-chevron-right']"))).click()
        time.sleep(4)
        driver.execute_script('window.scrollTo({}, {});'.format(coordinates['x'], coordinates['y']))
        time.sleep(4)
        
        pageSel = Selector(text = driver.page_source)

        urls = pageSel.css('div.card__content > a::attr(href)').extract()
        titles = title_test = pageSel.css('div.card__content > a > h5::text').extract()

        urldict['urls'] = urldict['urls'] + urls
        urldict['titles'] = urldict['titles']+ titles
    except (NoSuchElementException, WebDriverException, ElementNotInteractableException) as e:
        print("No 'Next' button found")
        break

driver.quit()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


No 'Next' button found


In [12]:
print(len(urldict['urls']),
     len(urldict['titles']))

231 231


In [13]:
oneurl = urldict['urls'][1]
oneurl

'/insights/retail-holiday-newsletter-strong-sales-and-winning-tips-for-2020s/'

In [14]:
main_url = 'https://www.bain.com'
urllist = list(map(lambda url: main_url + url, urldict['urls']))

In [15]:
len(urllist)

231

In [17]:
with open('../data_raw/urls/bain_urls.txt', 'w') as f:
    for url in urllist:
        f.write(url + "\n")

### Load URLs

In [18]:
bain_urls = []
with open('../data_raw/urls/bain_urls.txt', 'r') as f:
    for line in f:
        bain_urls.append(line.strip())

print(len(bain_urls),
      '\n',
     bain_urls[199])

231 
 https://www.bain.com/insights/brand-rethink-ceo-forum/


In [19]:
urllist = bain_urls

### Testing one page

In [20]:
page_url = urllist[0]

In [21]:
page_dict = dict()
main_dict = dict()

In [22]:
text_sel = '#body > article > div.article-container ::text'
title_sel = '//meta[contains(@property, "og:title")]/@content'
date_sel = '//meta[contains(@property, "article:published_time")]/@content'
datemod_sel = '//meta[contains(@property, "article:modified_time")]/@content'
texttype_sel = '//meta[contains(@property, "og:type")]/@content'

siteno = 1

page_html = requests.get(page_url).content
page_sel = Selector(text = page_html)

page_id = 'site' + str(siteno)
page_title = page_sel.xpath(title_sel).extract_first()
page_date = page_sel.xpath(date_sel).extract_first()
page_datemod = page_sel.xpath(datemod_sel).extract_first()
page_texttype = page_sel.xpath(texttype_sel).extract_first()
page_dldate = date.today().strftime('%Y/%m/%d')
page_text = page_sel.css(text_sel).extract()

page_text = ' '.join(page_text)
     
page_dict['url'] = page_url
page_dict['title'] = page_title
page_dict['article date'] = page_date
page_dict['modified date'] = page_datemod
page_dict['download date'] = page_dldate
page_dict['text type'] = page_texttype
page_dict['text'] = page_text

main_dict[page_id] = page_dict

In [26]:
len(main_dict)

1

In [27]:
urllist_snip = urllist[:4]
urllist_snip

['https://www.bain.com/insights/life-insurers-spread-their-wings-but-can-they-maintain-altitude/',
 'https://www.bain.com/insights/retail-holiday-newsletter-strong-sales-and-winning-tips-for-2020s/',
 'https://www.bain.com/insights/for-banks-modernizing-technology-has-become-a-perpetual-challenge/',
 'https://www.bain.com/insights/a-smarter-way-for-healthcare-companies-to-go-digital/']

In [28]:
page_dict = dict()
main_dict = dict()

In [29]:
len(main_dict)

0

In [30]:
from scrapy.crawler import CrawlerProcess
from datetime import date

# Create class
class bain_spider (scrapy.Spider):
    name = "bain_spider"
    
    def start_requests(self):
        urls = urllist
        
        for url in urls:
            yield scrapy.Request(url = url, callback = self.parse)
            
    #Parsing
    def parse(self, response):
        print(response.url)
        
        page_dict = dict()
        siteno = len(main_dict) + 1
        
        page_url = response.url
        
        page_html = requests.get(page_url).content
        page_sel = Selector(text = page_html)
        
        text_sel = '#body > article > div.article-container ::text'
        title_sel = '//meta[contains(@property, "og:title")]/@content'
        date_sel = '//meta[contains(@property, "article:published_time")]/@content'
        datemod_sel = '//meta[contains(@property, "article:modified_time")]/@content'
        texttype_sel = '//meta[contains(@property, "og:type")]/@content'
        
        page_title = page_sel.xpath(title_sel).extract_first()
        page_date = page_sel.xpath(date_sel).extract_first()
        page_datemod = page_sel.xpath(datemod_sel).extract_first()
        page_texttype = page_sel.xpath(texttype_sel).extract_first()
        page_dldate = date.today().strftime('%Y/%m/%d')
        page_text = page_sel.css(text_sel).extract()

        page_text = ' '.join(page_text)
        
        page_id = 'site' + str(siteno)
        
        page_dict['url'] = page_url
        page_dict['title'] = page_title
        page_dict['article date'] = page_date
        page_dict['modified date'] = page_datemod
        page_dict['download date'] = page_dldate
        page_dict['text type'] = page_texttype
        page_dict['text'] = page_text
        
        main_dict[page_id] = page_dict
        
# Initiatlize dictionary
main_dict = dict()

#Run spider
process = CrawlerProcess()
siteno = 0
process.crawl(bain_spider)
process.start()

2020-02-18 09:54:31 [scrapy.utils.log] INFO: Scrapy 1.5.2 started (bot: scrapybot)
2020-02-18 09:54:31 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.6.1, Platform Windows-10-10.0.18362-SP0
2020-02-18 09:54:31 [scrapy.crawler] INFO: Overridden settings: {}
2020-02-18 09:54:32 [scrapy.extensions.telnet] INFO: Telnet Password: 932173ed134ac9d7
2020-02-18 09:54:32 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-02-18 09:54:32 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloade

https://www.bain.com/insights/in-2020-asia-will-have-the-worlds-largest-gdp-wef/


2020-02-18 09:54:36 [urllib3.connectionpool] DEBUG: https://www.bain.com:443 "GET /insights/in-2020-asia-will-have-the-worlds-largest-gdp-wef/ HTTP/1.1" 200 39718
2020-02-18 09:54:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bain.com/insights/for-banks-modernizing-technology-has-become-a-perpetual-challenge/> (referer: None)
2020-02-18 09:54:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bain.com/insights/retail-holiday-newsletter-2019-issue-3-supply-chain/> (referer: None)
2020-02-18 09:54:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bain.com/insights/investors-have-soured-on-europes-banks/> (referer: None)
2020-02-18 09:54:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bain.com/insights/retail-holiday-newsletter-2019-issue-4-taking-stock-retailers-holiday-marketing-strategies/> (referer: None)
2020-02-18 09:54:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bain.com/insights/how-the-best-sales-operations-te

https://www.bain.com/insights/life-insurers-spread-their-wings-but-can-they-maintain-altitude/


2020-02-18 09:54:38 [urllib3.connectionpool] DEBUG: https://www.bain.com:443 "GET /insights/life-insurers-spread-their-wings-but-can-they-maintain-altitude/ HTTP/1.1" 200 45881
2020-02-18 09:54:38 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bain.com:443


...

2020-02-18 10:00:24 [urllib3.connectionpool] DEBUG: https://www.bain.com:443 "GET /insights/asia-discovers-its-m-and-a-potential/ HTTP/1.1" 200 41951
2020-02-18 10:00:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.bain.com/insights/helping-knowledge-management-be-all-it-can-be/> (referer: None)
2020-02-18 10:00:24 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bain.com:443


https://www.bain.com/insights/localization-revolution-in-consumer-markets/


2020-02-18 10:00:25 [urllib3.connectionpool] DEBUG: https://www.bain.com:443 "GET /insights/localization-revolution-in-consumer-markets/ HTTP/1.1" 200 37672
2020-02-18 10:00:25 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.bain.com:443


https://www.bain.com/insights/helping-knowledge-management-be-all-it-can-be/


2020-02-18 10:00:27 [urllib3.connectionpool] DEBUG: https://www.bain.com:443 "GET /insights/helping-knowledge-management-be-all-it-can-be/ HTTP/1.1" 200 46492
2020-02-18 10:00:27 [scrapy.core.engine] INFO: Closing spider (finished)
2020-02-18 10:00:27 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 101607,
 'downloader/request_count': 231,
 'downloader/request_method_count/GET': 231,
 'downloader/response_bytes': 10559056,
 'downloader/response_count': 231,
 'downloader/response_status_count/200': 231,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2020, 2, 18, 9, 0, 27, 243688),
 'log_count/DEBUG': 694,
 'log_count/INFO': 13,
 'response_received_count': 231,
 'scheduler/dequeued': 231,
 'scheduler/dequeued/memory': 231,
 'scheduler/enqueued': 231,
 'scheduler/enqueued/memory': 231,
 'start_time': datetime.datetime(2020, 2, 18, 8, 54, 33, 2378)}
2020-02-18 10:00:27 [scrapy.core.engine] INFO: Spider closed (finished)


In [31]:
process.stop()

<DeferredList at 0x2207c8a5cc0 current result: []>

In [32]:
len(main_dict)

231

In [33]:
import json

with open('../data_raw/articles/bain_articles.json', 'w') as file:
    json.dump(main_dict, file)